%load_ext autoreload
%autoreload 2

In [53]:
!pwd
import sys
import os
sys.path.append(os.path.abspath('../'))
import pandas as pd
from datetime import datetime

from data_helpers.syntax_graph import SyntaxGraph
from data_helpers.conllu_reader import CoNNLUReader

/Users/rabauti/repos/eki/prg1978__opikulausetekorpus/heete_loendid


# Tabel abiverbid 
da + võima, saama ja tohtima
ma + pidama

**Ülesande püstitus**
    
Kõik abiverbid, kus esineb tabelis [list_da.csv](list_da.csv) olev verb ja verbil on otsene alluv deprel=xcomp, feats sisaldab inf

1. id; 2. sentence; 3. verb (st võima, saama, tohtima, pidama); 4. head (st verbi ülemuse lemma); 5. level (keeletase); 6. sub (nagu enne).



In [54]:
%%time

# verbid etteantud nimekirjast
df_verbs = pd.read_csv('list_aux.csv')
my_verbs = list(df_verbs['lemma'].unique())


file_name = '../data/Model2Eesti-keele-kui-teise-keele-kooliõpikute-lausete-korpus-2021.conllu'
my_reader = CoNNLUReader(
   file_name = file_name
)

CPU times: user 1.47 ms, sys: 3.61 ms, total: 5.08 ms
Wall time: 4.18 ms


In [55]:
my_verbs

['võima', 'saama', 'tohtima', 'pidama']

In [65]:
%%time

date_time = datetime.now().strftime("%Y%m%d-%H%M%S")

collected_data = []
count = 0
for collection_id, graph in my_reader.get_sentences():
    # matrix for node distances
    dpath = graph.get_distances_matrix()
    
    verb_nodes = [v for v in graph.get_nodes_by_attributes(attrname="POS", attrvalue="VERB")]
    aux_nodes = [v for v in graph.get_nodes_by_attributes(attrname="deprel", attrvalue="aux") if graph.nodes[v]["lemma"] in my_verbs]
    if not len(aux_nodes): 
        continue
    
    
    # aux on tegusõna 'võima', 'saama', 'tohtima', 'pidama'
    for aux in aux_nodes:
        aux_head = graph.nodes[aux]["head"]
        if aux_head not in verb_nodes:
            continue
       
        kids = [k for k in dpath[aux_head] if dpath[aux_head][k] == 1]
       
        #graph.draw_graph2(highlight=[aux_head, kids])
        d = {
            'id':  graph.get_metadata('row_id'),
            'sentence':  graph.get_metadata('text'),
            'aux_head':  graph.nodes[aux_head]["lemma"],
            'aux':  graph.nodes[aux]["lemma"],
            'level':  graph.get_metadata('sent_level'),
            'sub': " ".join(
                        [graph.nodes[n]["form"] for n in sorted([aux_head] + kids)]
                    ),
            'sup': int( graph.nodes[aux_head]["feats"] is not None and "VerbForm" in graph.nodes[aux_head]["feats"] and graph.nodes[aux_head]["feats"]["VerbForm"] == 'Sup' ),
            'inf': int( graph.nodes[aux_head]["feats"] is not None and "VerbForm" in graph.nodes[aux_head]["feats"] and graph.nodes[aux_head]["feats"]["VerbForm"] == 'Inf' ),
            # 'aux_head_feats': graph.nodes[aux_head]["feats"],
        }
            
        collected_data.append(d)

df = pd.DataFrame.from_dict(collected_data)
df.to_csv(f'verb_aux_{date_time}.csv', index=None)
df.head(30)

../data/Model2Eesti-keele-kui-teise-keele-kooliõpikute-lausete-korpus-2021.conllu
CPU times: user 3.74 s, sys: 39.5 ms, total: 3.78 s
Wall time: 3.8 s


,id,sentence,aux_head,aux,level,sub,sup,inf,aux_head_feats
0,2,"Põhikool on kolmeastmeline, pärast üheksandat ...",minema,võima,gümn,", klassi võivad noored minna õppima",0,1,{'VerbForm': 'Inf'}
1,6,"Võib valida humanitaarkallakuga kooli, mida ka...",valima,võima,gümn,Võib valida kooli .,0,1,{'VerbForm': 'Inf'}
2,12,Ülikoolidest võib valida riikliku või eraüliko...,valima,võima,gümn,Ülikoolidest võib valida riikliku .,0,1,{'VerbForm': 'Inf'}
3,44,Uude seltskonda sattudes võiks leida alguses ü...,leidma,võima,gümn,sattudes võiks leida alguses inimese .,0,1,{'VerbForm': 'Inf'}
4,46,"Võiksid tähele panna, millest klassikaaslased ...",panema,võima,gümn,Võiksid tähele panna räägivad .,0,1,{'VerbForm': 'Inf'}
5,47,"Tuleks arvestada, et klass ei pea kohanema sin...",kohanema,pidama,gümn,", et klass ei pea kohanema sinuga",1,0,"{'Case': 'Ill', 'VerbForm': 'Sup', 'Voice': 'A..."
6,51,"Muidu võib juhtuda, et sa ei saa kooli lõpuski...",juhtuma,võima,gümn,Muidu võib juhtuda saa .,0,1,{'VerbForm': 'Inf'}
7,55,"Sulle võib imelik tunduda, et pool klassi sinu...",tunduma,võima,gümn,Sulle võib imelik tunduda naudib .,0,1,{'VerbForm': 'Inf'}
8,59,Ideaalses tunniplaanis lükatakse sinna vahele ...,puhkama,saama,gümn,", et pea saaks puhata",0,1,{'VerbForm': 'Inf'}
9,60,"Kirjanduses, kus õpilased peaksid palju raamat...",lugema,pidama,gümn,", kus õpilased peaksid raamatuid lugema kirjut...",1,0,"{'Case': 'Ill', 'VerbForm': 'Sup', 'Voice': 'A..."
